<a href="https://colab.research.google.com/github/grfaith/AmericanStories/blob/main/first_chunks_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [21]:
import os
import pandas as pd

# Specify the path to the output directory
output_dir = '/content/drive/My Drive/output'

# Get a list of all files in the directory matching the format output_chunk_N
files = [f for f in os.listdir(output_dir) if f.startswith('output_chunk_') and f.endswith('.csv')]

# Sort files based on the number N in output_chunk_N
files.sort(key=lambda f: int(f.split('_')[-1].split('.')[0]))

'''
# Initialize an empty list to hold the dataframes
dfs = []

# Iterate over the files and read them into dataframes
for i, file in enumerate(files):
    file_path = os.path.join(output_dir, file)

    # Read the first file with headers
    if i == 0:
        df = pd.read_csv(file_path, low_memory=False)  # Use the column names from the first file
    else:
        # Read subsequent files without headers
        df = pd.read_csv(file_path, header=None, low_memory=False)
        df.columns = dfs[0].columns  # Ensure the columns match

    dfs.append(df)

# Concatenate all the dataframes
concatenated_df = pd.concat(dfs, ignore_index=True)

# Display the result (or you can save it to a file)
print(concatenated_df)
'''

'\n# Initialize an empty list to hold the dataframes\ndfs = []\n\n# Iterate over the files and read them into dataframes\nfor i, file in enumerate(files):\n    file_path = os.path.join(output_dir, file)\n    \n    # Read the first file with headers\n    if i == 0:\n        df = pd.read_csv(file_path, low_memory=False)  # Use the column names from the first file\n    else:\n        # Read subsequent files without headers\n        df = pd.read_csv(file_path, header=None, low_memory=False)\n        df.columns = dfs[0].columns  # Ensure the columns match\n\n    dfs.append(df)\n\n# Concatenate all the dataframes\nconcatenated_df = pd.concat(dfs, ignore_index=True)\n\n# Display the result (or you can save it to a file)\nprint(concatenated_df)\n'

In [ ]:
###  ***Transplanted code from AS_Explore_Postprocess_For_Commit.ipynb ***
###  This code runs the light processing routines from American Stories on AS-format article text.
###  The file is loaded and saved locally.

In [6]:
# Installs

!pip install symspellpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 12.8 MB/s eta 0:00:00


In [7]:
#Imports
import io
import os
import json
import pandas as pd
from tqdm import tqdm
import tqdm as tq

In [8]:
# let's initialize the package
import pkg_resources
from symspellpy import SymSpell, Verbosity
import string

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
en_dict = pkg_resources.resource_filename('symspellpy', 'frequency_dictionary_en_82_765.txt')
sym_spell.load_dictionary(en_dict, term_index=0, count_index=1)

True

In [9]:
# This is code from AS and has functions for text correction and polishing.
# we now create a few functions that take care of the issues flagged above

# thsese two functions implement spelling corrections
def check_word(word):
  no_punc_word = word.strip(string.punctuation)
  if len(no_punc_word) > 0:
    suggestions = sym_spell.lookup(no_punc_word, Verbosity.CLOSEST, max_edit_distance=1, include_unknown=True, transfer_casing=True)
  else:
    return word
  return word.replace(no_punc_word, suggestions[0].term)

def spell_check(text):
  lines = text.split('\n')
  checked_lines = []
  for line in lines:
    words = line.split(' ')
    checked_line = ' '.join([check_word(word) for word in words])
    checked_lines.append(checked_line)
  return '\n'.join(checked_lines)

# this function checks capitalization
def capitalization_check(text):
  lines = text.split('\n')
  checked_lines = []
  for line in lines:
    words = line.split(' ')
    for i in range(1, len(words)):
      if words[i-1][-1] in ['.', '!', '?']:
        words[i] = words[i].capitalize()
      else:
        no_punc_word = words[i].strip(string.punctuation)
        if no_punc_word in sym_spell.words and no_punc_word not in ['i', "i'll"]: # Check that the word is not a propper noun
          words[i] = words[i].replace(no_punc_word, no_punc_word.lower())

    checked_lines.append(' '.join(words))
  return '\n'.join(checked_lines)

# this functions corrects line breaks
def line_merge(text):
  lines = [l.split() for l in text.split('\n')]
  for i in range(len(lines) - 1):
    if len(lines[i]) == 0 or len(lines[i+1]) == 0:
      continue
    elif lines[i][-1][-1] == '-': # Automatically merge if a line ends with a hyphen
      lines[i][-1] = lines[i][-1][:-1] + lines[i+1][0]
      lines[i+1] = lines[i+1][1:]
    elif lines[i][-1].strip(string.punctuation).lower() not in sym_spell.words or lines[i+1][0].strip(string.punctuation).lower() not in sym_spell.words:
      if (lines[i][-1].strip(string.punctuation).lower() + lines[i+1][0].strip(string.punctuation).lower()) in sym_spell.words:
        lines[i][-1] += lines[i+1][0]
        lines[i+1] = lines[i+1][1:]

  return '\n'.join([' '.join(l) for l in lines])


In [10]:
# this functions implements all three methods
def postprocess(text):
  merged = line_merge(text)
  checked = spell_check(merged)
  capitalization_normalized = capitalization_check(checked)
  return capitalization_normalized

In [28]:
import os
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# Specify the path to the output directory
output_dir = '/content/drive/My Drive/output'

# Get a list of all files in the directory matching the format output_chunk_N
files = [f for f in os.listdir(output_dir) if f.startswith('output_chunk_') and f.endswith('.csv')]

# Sort files based on the number N in output_chunk_N and remove any duplicates using set
files = sorted(list(set(files)), key=lambda f: int(f.split('_')[-1].split('.')[0]))

# Ensure 'postprocess' function is defined
def postprocess(article):
    # Example postprocess logic (replace this with your actual logic)
    return article.lower()  # Dummy example: converting to lowercase

# Define a function to process each file
def process_file(file):
    file_path = os.path.join(output_dir, file)

    # Read the file
    df = pd.read_csv(file_path, low_memory=False)

    # Apply the postprocess function to the 'article_text' column
    df['article_p'] = df['article_text'].apply(lambda x: postprocess(x) if not pd.isnull(x) else x)

    # Save the processed file (you can overwrite the file or create a new one)
    output_file_path = os.path.join(output_dir, f'processed_{file}')  # Change to your preferred output
    df.to_csv(output_file_path, index=False)

    print(f'{file} has been processed and saved to {output_file_path}')

# Use ThreadPoolExecutor to process files in parallel
with ThreadPoolExecutor() as executor:
    # Submit each file for processing
    executor.map(process_file, files)

print("All files have been processed.")


output_chunk_6.csv has been processed and saved to /content/drive/My Drive/output/processed_output_chunk_6.csv
output_chunk_5.csv has been processed and saved to /content/drive/My Drive/output/processed_output_chunk_5.csv
output_chunk_4.csv has been processed and saved to /content/drive/My Drive/output/processed_output_chunk_4.csv
output_chunk_3.csv has been processed and saved to /content/drive/My Drive/output/processed_output_chunk_3.csv
output_chunk_2.csv has been processed and saved to /content/drive/My Drive/output/processed_output_chunk_2.csv
output_chunk_11.csv has been processed and saved to /content/drive/My Drive/output/processed_output_chunk_11.csv
output_chunk_1.csv has been processed and saved to /content/drive/My Drive/output/processed_output_chunk_1.csv
output_chunk_7.csv has been processed and saved to /content/drive/My Drive/output/processed_output_chunk_7.csv
output_chunk_8.csv has been processed and saved to /content/drive/My Drive/output/processed_output_chunk_8.csv

In [26]:
check_df = pd.read_csv('/content/drive/My Drive/output/processed_output_chunk_1.csv')
print (check_df.head())


                                          article_ID  row_number  aether  \
0  6_1799-01-28_p4_sn83045634_00332895059_1799012...         107       0   
1  8_1799-04-15_p4_sn83045634_00332895059_1799041...         316       0   
2  5_1802-05-31_p1_sn83016063_00332895060_1802053...         152       0   
3  13_1802-07-12_p4_sn83016063_00332895060_180207...         148       0   
4  21_1803-05-30_p4_sn83016072_00332895060_180305...         384       0   

   asteroid  aurora  borealis  celestial  comet  copernicus  cosmic  ...  \
0         0       0         0          0      0           0       0  ...   
1         0       0         0          0      0           0       0  ...   
2         0       3         0          0      0           0       0  ...   
3         0       0         0          0      0           0       0  ...   
4         0       0         0          0      0           0       0  ...   

   uranus  venus  venus transit  total_keyword_count    pub_date  pub_year  \
0       